In [38]:
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import torch
import torch.nn as nn
import torch.nn.functional
import torchvision
from torch.utils.data import TensorDataset
import math

In [39]:
def load_image(path, size):
    image = cv2.imread(path)
    image = cv2.resize(image, (size,size))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = image/255.
    return image

def load_data(root_path, size):
    images = []
    masks = []
    x = 0
    for path in sorted(glob(root_path)):
        img = load_image(path, size)
        if 'mask' in path:
            if x:
                masks[-1] += img
                masks[-1] = np.array(masks[-1]>0.5, dtype='float64')
            else:
                masks.append(img)
                x = 1
        else:
            images.append(img)
            x = 0
    return np.array(images), np.array(masks)

## Loading image and mask data

In [40]:
import cv2
import numpy as np
from glob import glob
import os

def load_image(path, size):
    image = cv2.imread(path)
    image = cv2.resize(image, (size, size))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = image / 255.0
    return image

def load_data(image_folder, mask_folder, size):
    images = []
    masks = []
    
    # Get sorted file lists
    image_paths = sorted(glob(os.path.join(image_folder, '*')))
    mask_paths = sorted(glob(os.path.join(mask_folder, '*')))
    
    for img_path, mask_path in zip(image_paths, mask_paths):
        # Load image
        img = load_image(img_path, size)
        images.append(img)
        
        # Load mask
        mask = load_image(mask_path, size)
        masks.append(mask)
    
    return np.array(images), np.array(masks)


In [33]:
image_path = "Your designated Image path"
masks_path = "Your designated mask path"

size = 224
X, Y = load_data(image_path, masks_path, size)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, shuffle=True)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size=0.5,shuffle=True)
X_train.shape, X_valid.shape, X_test.shape, Y_train.shape, Y_valid.shape, Y_test.shape

((5474, 224, 224),
 (1173, 224, 224),
 (1173, 224, 224),
 (5474, 224, 224),
 (1173, 224, 224),
 (1173, 224, 224))

In [35]:
X_train = np.expand_dims(X_train,-1)
Y_train = np.expand_dims(Y_train,-1)

def elastic_transform(image, mask, alpha_affine):

    random_state = np.random.RandomState(None)
    shape = image.shape
    shape_size = shape[:2]

    # Random affine
    center_square = np.float32(shape_size) // 2
    square_size = 42#min(shape_size) // 3
    pts1 = np.float32([center_square + square_size, [center_square[0]+square_size, center_square[1]-square_size], center_square - square_size])
    pts2 = pts1 + random_state.uniform(-alpha_affine, alpha_affine, size=pts1.shape).astype(np.float32)
    M = cv2.getAffineTransform(pts1, pts2)
    image = cv2.warpAffine(image, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    mask = cv2.warpAffine(mask, M, shape_size[::-1], borderMode=cv2.BORDER_REFLECT_101)
    return image, mask

def hflip_transform(image, mask):
    image = cv2.flip(image, 1)
    mask = cv2.flip(mask, 1)
    return image, mask

def vflip_transform(image, mask):
    image = cv2.flip(image, 0)
    mask = cv2.flip(mask, 0)
    return image, mask

def flip_transform(image, mask):
    image = cv2.flip(image, -1)
    mask = cv2.flip(mask, -1)
    return image, mask

el_train_images = [elastic_transform(img, mask, img.shape[1] * 0.04) for img,mask in zip(X_train,Y_train )]

hflip_train_images = [hflip_transform(img, mask) for img,mask in zip(X_train,Y_train )]

vflip_train_images = [vflip_transform(img, mask) for img,mask in zip(X_train,Y_train )]

flip_train_images = [flip_transform(img, mask) for img,mask in zip(X_train,Y_train )]


train_images_list = [el_train_images, hflip_train_images, vflip_train_images, flip_train_images]


for each_el_tf in train_images_list:
    el_tf_imgs,  el_tf_masks = zip(*each_el_tf)
    el_tf_imgs = list(el_tf_imgs)
    el_tf_masks = list(el_tf_masks)
    X_train = np.concatenate((X_train, np.expand_dims(el_tf_imgs, -1)))
    Y_train = np.concatenate((Y_train, np.expand_dims(el_tf_masks, -1)))

X_train = torch.from_numpy(np.float32(np.squeeze(X_train,-1))).unsqueeze(1)
Y_train = torch.from_numpy(np.float32(np.squeeze(Y_train,-1))).unsqueeze(1)
X_valid = torch.from_numpy(np.float32(X_valid)).unsqueeze(1)
Y_valid = torch.from_numpy(np.float32(Y_valid)).unsqueeze(1)
X_test = torch.from_numpy(np.float32(X_test)).unsqueeze(1)
Y_test = torch.from_numpy(np.float32(Y_test)).unsqueeze(1)
X_train.shape, X_valid.shape, X_test.shape, Y_train.shape, Y_valid.shape, Y_test.shape

(torch.Size([27370, 1, 224, 224]),
 torch.Size([1173, 1, 224, 224]),
 torch.Size([1173, 1, 224, 224]),
 torch.Size([27370, 1, 224, 224]),
 torch.Size([1173, 1, 224, 224]),
 torch.Size([1173, 1, 224, 224]))

In [36]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(TensorDataset(X_train,Y_train), batch_size=BATCH_SIZE,
                                        shuffle=True)

valid_loader = torch.utils.data.DataLoader(TensorDataset(X_valid,Y_valid), batch_size=BATCH_SIZE,
                                        shuffle=True)

test_loader = torch.utils.data.DataLoader(TensorDataset(X_test,Y_test), batch_size=BATCH_SIZE,
                                        shuffle=True)

In [44]:
def window_partition(x, window_size):
    B, H, W, C = x.shape
    x = x.view(B, H // window_size[0], window_size[0], W // window_size[1], window_size[1], C)
    windows = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, window_size[0], window_size[1], C)
    return windows

def window_reverse(windows, window_size, H, W):
    C = windows.shape[-1]
    x = windows.view(-1, H // window_size[0], W // window_size[1], window_size[0], window_size[1], C)
    x = x.permute(0, 1, 3, 2, 4, 5).contiguous().view(-1, H, W, C)
    return x

def get_relative_position_index(win_h: int, win_w: int):
    # get pair-wise relative position index for each token inside the window
    coords = torch.stack(torch.meshgrid(torch.arange(win_h), torch.arange(win_w),indexing='ij'))  # 2, Wh, Ww
    coords_flatten = torch.flatten(coords, 1)  # 2, Wh*Ww
    relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]  # 2, Wh*Ww, Wh*Ww
    relative_coords = relative_coords.permute(1, 2, 0).contiguous()  # Wh*Ww, Wh*Ww, 2
    relative_coords[:, :, 0] += win_h - 1  # shift to start from 0
    relative_coords[:, :, 1] += win_w - 1
    relative_coords[:, :, 0] *= 2 * win_w - 1
    return relative_coords.sum(-1)  # Wh*Ww, Wh*Ww

class WindowAttention(nn.Module):
    def __init__(
            self,
            dim,
            window_size,
    ):
        super().__init__()
        self.window_size = window_size
        self.window_area = self.window_size[0]*self.window_size[1]
        self.num_heads = 4
        head_dim =  dim // self.num_heads
        # attn_dim = head_dim * self.num_heads
        self.scale = head_dim ** -0.5

        self.relative_position_bias_table = nn.Parameter(torch.zeros((2 * window_size[0] - 1) **2, self.num_heads))

        # get pair-wise relative position index for each token inside the window
        self.register_buffer("relative_position_index", get_relative_position_index(self.window_size[0], self.window_size[1]), persistent=False)

        self.qkv = nn.Linear(dim, dim * 3)
        self.proj = nn.Linear(dim, dim)

        torch.nn.init.trunc_normal_(self.relative_position_bias_table, std=.02)
        self.softmax = nn.Softmax(dim=-1)

    def _get_rel_pos_bias(self):
        relative_position_bias = self.relative_position_bias_table[
            self.relative_position_index.view(-1)].view(self.window_area, self.window_area, -1)  # Wh*Ww,Wh*Ww,nH
        relative_position_bias = relative_position_bias.permute(2, 0, 1).contiguous()  # nH, Wh*Ww, Wh*Ww
        return relative_position_bias.unsqueeze(0)

    def forward(self, x, mask = None):
        B_, N, C = x.shape
        qkv = self.qkv(x).reshape(B_, N, 3, self.num_heads, -1).permute(2, 0, 3, 1, 4)
        q, k, v = qkv.unbind(0)


        q = q * self.scale
        attn = q @ k.transpose(-2, -1)
        attn = attn + self._get_rel_pos_bias()
        if mask is not None:
            num_win = mask.shape[0]
            attn = attn.view(-1, num_win, self.num_heads, N, N) + mask.unsqueeze(1).unsqueeze(0)
            attn = attn.view(-1, self.num_heads, N, N)
        attn = self.softmax(attn)
        x = attn @ v

        x = x.transpose(1, 2).reshape(B_, N, -1)
        x = self.proj(x)
        return x

In [45]:
class SwinTransformerBlock(nn.Module):
    def __init__(
            self,  dim, input_resolution, window_size = 7, shift_size = 0):

        super().__init__()
        self.input_resolution = input_resolution
        window_size = (window_size, window_size)
        shift_size = (shift_size, shift_size)
        self.window_size = window_size
        self.shift_size = shift_size
        self.window_area = self.window_size[0] * self.window_size[1]

        self.norm1 = nn.LayerNorm(dim)
        self.attn = WindowAttention(
            dim,
            window_size=self.window_size,
        )

        self.norm2 = nn.LayerNorm(dim)

        self.mlp = nn.Sequential(
            nn.Linear(dim, 4 * dim),
            nn.GELU(),
            nn.LayerNorm(4 * dim),
            nn.Linear( 4 * dim, dim)
        )

        if self.shift_size:
            # calculate attention mask for SW-MSA
            H, W = self.input_resolution
            H = math.ceil(H / self.window_size[0]) * self.window_size[0]
            W = math.ceil(W / self.window_size[1]) * self.window_size[1]
            img_mask = torch.zeros((1, H, W, 1))  # 1 H W 1
            cnt = 0
            for h in (
                    slice(0, -self.window_size[0]), 
                    slice(-self.window_size[0], -self.shift_size[0]),
                    slice(-self.shift_size[0], None)):
                for w in (
                        slice(0, -self.window_size[1]),
                        slice(-self.window_size[1], -self.shift_size[1]),
                        slice(-self.shift_size[1], None)):
                    img_mask[:, h, w, :] = cnt
                    cnt += 1
            mask_windows = window_partition(img_mask, self.window_size)  # nW, window_size, window_size, 1
            mask_windows = mask_windows.view(-1, self.window_area)
            attn_mask = mask_windows.unsqueeze(1) - mask_windows.unsqueeze(2)
            attn_mask = attn_mask.masked_fill(attn_mask != 0, float(-100.0)).masked_fill(attn_mask == 0, float(0.0))
        else:
            attn_mask = None

        self.register_buffer("attn_mask", attn_mask, persistent=False)

    def _attn(self, x):
        B, H, W, C = x.shape

        # cyclic shift
        if self.shift_size:
            shifted_x = torch.roll(x, shifts=(-self.shift_size[0], -self.shift_size[1]), dims=(1, 2))
        else:
            shifted_x = x

        # partition windows
        x_windows = window_partition(shifted_x, self.window_size)  # nW*B, window_size, window_size, C
        x_windows = x_windows.view(-1, self.window_area, C)  # nW*B, window_size*window_size, C

        # W-MSA/SW-MSA
        attn_windows = self.attn(x_windows, mask=self.attn_mask)  # nW*B, window_size*window_size, C

        # merge windows
        attn_windows = attn_windows.view(-1, self.window_size[0], self.window_size[1], C)
        shifted_x = window_reverse(attn_windows, self.window_size, H, W)  # B H' W' C
        shifted_x = shifted_x[:, :H, :W, :].contiguous()

        # reverse cyclic shift
        if self.shift_size:
            x = torch.roll(shifted_x, shifts=self.shift_size, dims=(1, 2))
        else:
            x = shifted_x
        return x

    def forward(self, x):
        B, H, W, C = x.shape
        B, H, W, C = x.shape
        x = x + self._attn(self.norm1(x))
        x = x.reshape(B, -1, C)
        x = x + self.mlp(self.norm2(x))
        x = x.reshape(B, H,W, C)
        return x

In [46]:
class PatchEmbedding(nn.Module):
    def __init__(self, in_ch, num_feat, patch_size):
        super().__init__()
        self.conv = nn.Conv2d(in_ch,num_feat, kernel_size=patch_size,
                                  stride=patch_size)

    def forward(self, X):
        # Output shape: (batch size, no. of patches, no. of channels)
        return self.conv(X).permute(0,2,3,1)

class PatchMerging(nn.Module):

    def __init__(
            self,
            dim
    ):
        super().__init__()
        self.norm = nn.LayerNorm(4 * dim)
        self.reduction = nn.Linear(4*dim, 2*dim, bias=False)

    def forward(self, x):
        B, H, W, C = x.shape
        x = x.reshape(B, H // 2, 2, W // 2, 2, C).permute(0, 1, 3, 4, 2, 5).flatten(3)
        x = self.norm(x)
        x = self.reduction(x)
        return x

class PatchExpansion(nn.Module):

    def __init__(
            self,
            dim
    ):
        super().__init__()
        self.norm = nn.LayerNorm(dim//2)
        self.expand = nn.Linear(dim, 2*dim, bias=False)

    def forward(self, x):

        x = self.expand(x)
        B, H, W, C = x.shape

        x = x.view(B, H , W, 2, 2, C//4)
        x = x.permute(0,1,3,2,4,5)

        x = x.reshape(B,H*2, W*2 , C//4)

        x = self.norm(x)
        return x

class FinalPatchExpansion(nn.Module):

    def __init__(
            self,
            dim
    ):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.expand = nn.Linear(dim, 16*dim, bias=False)

    def forward(self, x):

        x = self.expand(x)
        B, H, W, C = x.shape

        x = x.view(B, H , W, 4, 4, C//16)
        x = x.permute(0,1,3,2,4,5)

        x = x.reshape(B,H*4, W*4 , C//16)

        x = self.norm(x)
        return x

In [47]:
class SwinBlock(nn.Module):
    def __init__(self, dims, ip_res, ss_size = 3):
        super().__init__()
        self.swtb1 = SwinTransformerBlock(dim=dims, input_resolution=ip_res)
        self.swtb2 = SwinTransformerBlock(dim=dims, input_resolution=ip_res, shift_size=ss_size)

    def forward(self, x):
        return self.swtb2(self.swtb1(x))


class Encoder(nn.Module):
    def __init__(self, C, partioned_ip_res, num_blocks=3):
        super().__init__()
        H,W = partioned_ip_res[0], partioned_ip_res[1]
        self.enc_swin_blocks = nn.ModuleList([
            SwinBlock(C, (H, W)),
            SwinBlock(2*C, (H//2, W//2)),
            SwinBlock(4*C, (H//4, W//4))
        ])
        self.enc_patch_merge_blocks = nn.ModuleList([
            PatchMerging(C),
            PatchMerging(2*C),
            PatchMerging(4*C)
        ])

    def forward(self, x):
        skip_conn_ftrs = []
        for swin_block,patch_merger in zip(self.enc_swin_blocks, self.enc_patch_merge_blocks):
            x = swin_block(x)
            skip_conn_ftrs.append(x)
            x = patch_merger(x)
        return x, skip_conn_ftrs


class Decoder(nn.Module):
    def __init__(self, C, partioned_ip_res, num_blocks=3):
        super().__init__()
        H,W = partioned_ip_res[0], partioned_ip_res[1]
        self.dec_swin_blocks = nn.ModuleList([
            SwinBlock(4*C, (H//4, W//4)),
            SwinBlock(2*C, (H//2, W//2)),
            SwinBlock(C, (H, W))
        ])
        self.dec_patch_expand_blocks = nn.ModuleList([
            PatchExpansion(8*C),
            PatchExpansion(4*C),
            PatchExpansion(2*C)
        ])
        self.skip_conn_concat = nn.ModuleList([
            nn.Linear(8*C, 4*C),
            nn.Linear(4*C, 2*C),
            nn.Linear(2*C, 1*C)
        ])

    def forward(self, x, encoder_features):
        for patch_expand,swin_block, enc_ftr, linear_concatter in zip(self.dec_patch_expand_blocks, self.dec_swin_blocks, encoder_features,self.skip_conn_concat):
            x = patch_expand(x)
            x = torch.cat([x, enc_ftr], dim=-1)
            x = linear_concatter(x)
            x = swin_block(x)
        return x


class SwinUNet(nn.Module):
    def __init__(self, H, W, ch, C, num_class, num_blocks=3, patch_size = 4):
        super().__init__()
        self.patch_embed = PatchEmbedding(ch, C, patch_size)
        self.encoder = Encoder(C, (H//patch_size, W//patch_size),num_blocks)
        self.bottleneck = SwinBlock(C*(2**num_blocks), (H//(patch_size* (2**num_blocks)), W//(patch_size* (2**num_blocks))))
        self.decoder = Decoder(C, (H//patch_size, W//patch_size),num_blocks)
        self.final_expansion = FinalPatchExpansion(C)
        self.head        = nn.Conv2d(C, num_class, 1,padding='same')

    def forward(self, x):
        x = self.patch_embed(x)

        x,skip_ftrs  = self.encoder(x)

        x = self.bottleneck(x)

        x = self.decoder(x, skip_ftrs[::-1])

        x = self.final_expansion(x)

        x = self.head(x.permute(0,3,1,2))

        return x

In [58]:
import os
import torch
import numpy as np
import matplotlib.pyplot as plt

SAVE_DIR = "Your Directory"
os.makedirs(SAVE_DIR, exist_ok=True)

def train_epoch(model, dataloader, optimizer, loss_fn, device):
    model.train()
    losses = []
    for x, y in dataloader:
        optimizer.zero_grad()
        out = model(x.to(device))
        loss = loss_fn(out, y.to(device))
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    return np.mean(losses)

def validate_epoch(model, dataloader, loss_fn, device):
    model.eval()
    losses = []
    dice_scores = []
    with torch.no_grad():
        for x, y in dataloader:
            out = model(x.to(device))
            loss = loss_fn(out, y.to(device))
            losses.append(loss.item())
            
            # Calculate Dice coefficient
            pred = (out > 0.5).float()
            intersection = (pred * y.to(device)).sum()
            dice_score = (2. * intersection) / (pred.sum() + y.to(device).sum() + 1e-6)
            dice_scores.append(dice_score.item())
    return np.mean(losses), np.mean(dice_scores)

def plot_metrics(train_losses, val_losses, dice_scores, save_dir):
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(range(1, len(train_losses) + 1), train_losses, label='Train Loss')
    plt.plot(range(1, len(val_losses) + 1), val_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(range(1, len(dice_scores) + 1), dice_scores, label='Dice Coefficient', color='green')
    plt.xlabel('Epochs')
    plt.ylabel('Dice Coefficient')
    plt.title('Validation Dice Coefficient')
    plt.legend()

    plt.tight_layout()

    plots_save_path = "your_path"
    plot_path = os.path.join(save_dir, plots_save_path)
    plt.savefig(plot_path)
    print(f"Metrics plot saved to: {plot_path}")
    plt.close()

def train(model, train_loader, valid_loader, optimizer, loss_fn, epochs, min_epochs, early_stop_count, device):
    train_losses = []
    val_losses = []
    dice_scores = []
    best_valid_loss = float('inf')
    EARLY_STOP = early_stop_count

    for ep in range(1, epochs + 1):
        train_loss = train_epoch(model, train_loader, optimizer, loss_fn, device)
        valid_loss, dice_score = validate_epoch(model, valid_loader, loss_fn, device)

        train_losses.append(train_loss)
        val_losses.append(valid_loss)
        dice_scores.append(dice_score)

        print(f"Epoch: {ep}: train_loss={train_loss:.5f}, valid_loss={valid_loss:.5f}, dice_score={dice_score:.5f}")

        if ep % 10 == 0:
            save_path = os.path.join(SAVE_DIR, f"model_epoch_{ep}.pth")
            torch.save({
                'epoch': ep,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
            }, save_path)
            print(f"Model checkpoint saved at: {save_path}")

        if ep > min_epochs:
            if valid_loss < best_valid_loss:
                best_valid_loss = valid_loss
                best_model_path = os.path.join(SAVE_DIR, "best_model.pth")
                torch.save({
                    'epoch': ep,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                }, best_model_path)
                print(f"Best model saved with validation loss {valid_loss:.5f} at epoch {ep}")
                EARLY_STOP = early_stop_count
            else:
                EARLY_STOP -= 1
                if EARLY_STOP <= 0:
                    print("Early stopping triggered.")
                    plot_metrics(train_losses, val_losses, dice_scores, SAVE_DIR)
                    return train_losses, val_losses, dice_scores

    # Save plots after the last epoch
    plot_metrics(train_losses, val_losses, dice_scores, SAVE_DIR)
    return train_losses, val_losses, dice_scores


In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)
model = SwinUNet(224,224,1,32,1,3,4).to(DEVICE)

for p in model.parameters():
    if p.dim() > 1:
            nn.init.kaiming_uniform_(p)

optimizer = torch.optim.Adam(model.parameters())
loss_fn = nn.BCEWithLogitsLoss()

train(
    model=model,
    train_loader=train_loader,
    valid_loader=valid_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=40,
    min_epochs=5,
    early_stop_count=10,
    device=DEVICE
)

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(3,4, figsize=(10,8))
with torch.no_grad():
    for i in range(3):
        x_og,y_og = next(iter(train_loader))
        x = x_og[0]
        y = y_og[0]
        
        ax[i,0].imshow(x.squeeze(0).squeeze(0), cmap='gray')
        ax[i,0].set_title('Image')
        ax[i,1].imshow(y.squeeze(0).squeeze(0), cmap='gray')
        ax[i,1].set_title('Mask')
        x_og = x_og.to(DEVICE)
        out = model(x_og[:1])
        out = nn.Sigmoid()(out)
        out = out.squeeze(0).squeeze(0).cpu()
        ax[i,2].imshow(out, cmap='gray')
        ax[i,2].set_title('Prediction')
        ax[i,3].imshow((out>0.5).float(), cmap='gray')
        ax[i,3].set_title('Threshold Prediction')
plt.show()

## Dataset Testing for metrics calculation

In [48]:
from glob import glob
import numpy as np
from sklearn.model_selection import train_test_split
import cv2
import torch
import torch.nn as nn
import torch.nn.functional
import torchvision
from torch.utils.data import TensorDataset
import math

In [49]:
def load_image(path, size):
    image = cv2.imread(path)
    image = cv2.resize(image, (size,size))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = image/255.
    return image

def load_data(root_path, size):
    images = []
    masks = []
    x = 0
    for path in sorted(glob(root_path)):
        img = load_image(path, size)
        if 'mask' in path:
            if x:
                masks[-1] += img
                masks[-1] = np.array(masks[-1]>0.5, dtype='float64')
            else:
                masks.append(img)
                x = 1
        else:
            images.append(img)
            x = 0
    return np.array(images), np.array(masks)

In [50]:
import cv2
import numpy as np
from glob import glob
import os

def load_image(path, size):
    image = cv2.imread(path)
    image = cv2.resize(image, (size, size))
    image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    image = image / 255.0
    return image

def load_data(image_folder, mask_folder, size):
    images = []
    masks = []
    
    image_paths = sorted(glob(os.path.join(image_folder, '*')))
    mask_paths = sorted(glob(os.path.join(mask_folder, '*')))
    
    for img_path, mask_path in zip(image_paths, mask_paths):
        img = load_image(img_path, size)
        images.append(img)
        
        mask = load_image(mask_path, size)
        masks.append(mask)
    
    return np.array(images), np.array(masks)

In [90]:
image_path = "Your designated mask path"
masks_path = "Your designated mask path"

size = 224
X, Y = load_data(image_path, masks_path, size)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, shuffle=True)
X_valid, X_test, Y_valid, Y_test = train_test_split(X_test, Y_test, test_size=0.5,shuffle=True)
X_train.shape, X_valid.shape, X_test.shape, Y_train.shape, Y_valid.shape, Y_test.shape

((6732, 224, 224),
 (1443, 224, 224),
 (1443, 224, 224),
 (6732, 224, 224),
 (1443, 224, 224),
 (1443, 224, 224))

In [91]:
X_train = np.expand_dims(X_train,-1)
Y_train = np.expand_dims(Y_train,-1)

X_train = torch.from_numpy(np.float32(np.squeeze(X_train,-1))).unsqueeze(1)
Y_train = torch.from_numpy(np.float32(np.squeeze(Y_train,-1))).unsqueeze(1)

X_valid = torch.from_numpy(np.float32(X_valid)).unsqueeze(1)
Y_valid = torch.from_numpy(np.float32(Y_valid)).unsqueeze(1)
X_test = torch.from_numpy(np.float32(X_test)).unsqueeze(1)
Y_test = torch.from_numpy(np.float32(Y_test)).unsqueeze(1)

X_train.shape, X_valid.shape, X_test.shape, Y_train.shape, Y_valid.shape, Y_test.shape

(torch.Size([6732, 1, 224, 224]),
 torch.Size([1443, 1, 224, 224]),
 torch.Size([1443, 1, 224, 224]),
 torch.Size([6732, 1, 224, 224]),
 torch.Size([1443, 1, 224, 224]),
 torch.Size([1443, 1, 224, 224]))

In [92]:
BATCH_SIZE = 32

train_loader = torch.utils.data.DataLoader(TensorDataset(X_train,Y_train), batch_size=BATCH_SIZE,
                                        shuffle=True)

valid_loader = torch.utils.data.DataLoader(TensorDataset(X_valid,Y_valid), batch_size=BATCH_SIZE,
                                        shuffle=True)

test_loader = torch.utils.data.DataLoader(TensorDataset(X_test,Y_test), batch_size=BATCH_SIZE,
                                        shuffle=True)

## Prediction

In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import jaccard_score, f1_score, accuracy_score
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch.nn as nn

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")


checkpoint = torch.load("Your Checkpoint.pth", map_location=DEVICE)

state_dict = checkpoint['model_state_dict'] if 'model_state_dict' in checkpoint else checkpoint

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith("module."):
        new_state_dict[k[7:]] = v
    else:
        new_state_dict[k] = v


model = SwinUNet(224, 224, 1, 32, 1, 3, 4).to(DEVICE)

model.load_state_dict(new_state_dict)

model.to(DEVICE)
model.eval()  


class CustomDataset(Dataset):
    def __init__(self, image_folder, mask_folder=None, size=224):
        self.image_paths = sorted(glob(os.path.join(image_folder, '*')))
        self.mask_paths = sorted(glob(os.path.join(mask_folder, '*'))) if mask_folder else None
        self.size = size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.resize(image, (self.size, self.size))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image = image / 255.0
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0)  
        
        if self.mask_paths:
            mask_path = self.mask_paths[idx]
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            mask = cv2.resize(mask, (self.size, self.size))
            mask = mask / 255.0
            mask = torch.tensor(mask, dtype=torch.float32).unsqueeze(0)
            return image, mask
        
        return image
tput_folder ="Output folder path"  
os.makedirs(output_folder, exist_ok=True)


test_dataset = CustomDataset(image_folder, size=224)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

with torch.no_grad():
    for batch_idx, images in enumerate(test_loader):
        images = images.to(DEVICE)
        
        outputs = model(images)
        outputs = nn.Sigmoid()(outputs)  
        thresholded_outputs = (outputs > 0.5).float() 
        
        for i in range(images.size(0)):
            output_image = thresholded_outputs[i].cpu().squeeze(0)  
            save_path = os.path.join(output_folder, f"result_{batch_idx * test_loader.batch_size + i}.png")
            save_image(output_image, save_path)

            print(f"Saved: {save_path}")

print("Segmentation results saved successfully!")
image_folder = "Image Folder path"  
ou


In [ ]:
import torch
import torch.nn.functional as F
from sklearn.metrics import jaccard_score, f1_score, accuracy_score
import numpy as np
from collections import OrderedDict
import matplotlib.pyplot as plt
import torch.nn as nn


DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {DEVICE}")

checkpoint = torch.load("Your model checkpoint.pth", map_location=DEVICE)

state_dict = checkpoint['model_state_dict'] if 'model_state_dict' in checkpoint else checkpoint

new_state_dict = OrderedDict()
for k, v in state_dict.items():
    if k.startswith("module."):
        new_state_dict[k[7:]] = v 
    else:
        new_state_dict[k] = v

model = SwinUNet(224, 224, 1, 32, 1, 3, 4).to(DEVICE)

model.load_state_dict(new_state_dict)

model.to(DEVICE)
model.eval()  

class CustomDataset(Dataset):
    def __init__(self, image_folder, size=224):
        self.image_paths = sorted(glob(os.path.join(image_folder, '*')))
        self.size = size

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = cv2.imread(img_path)
        image = cv2.resize(image, (self.size, self.size))
        image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
        image = image / 255.0
        image = torch.tensor(image, dtype=torch.float32).unsqueeze(0) 
        return image, os.path.basename(img_path)  

image_folder = "Image folder"
output_folder ="Output folder"  
os.makedirs(output_folder, exist_ok=True)


test_dataset = CustomDataset(image_folder, size=224)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

with torch.no_grad():
    for batch_idx, (images, filenames) in enumerate(test_loader):
        images = images.to(DEVICE)
        
        outputs = model(images)
        outputs = nn.Sigmoid()(outputs)  
        thresholded_outputs = (outputs > 0.5).float()  
        
        for i in range(images.size(0)):
            output_image = thresholded_outputs[i].cpu().squeeze(0)  
            save_path = os.path.join(output_folder, filenames[i])  
            save_image(output_image, save_path)

            print(f"Saved: {save_path}")

print("Segmentation results saved successfully!")
